In [15]:
import pandas as pd
import pyodbc
from IPython.display import display, clear_output

# 1) Cargar y limpiar el DataFrame
df = pd.read_excel(
    r'C:\Users\rrs23\OneDrive\Documentos\OneDrive\Documentos\Datos\BaseQ.xlsx',
    sheet_name='Base Principal'
)

# Limpio espacios y renombro solo la columna 'Correo Cliente'
df.columns = df.columns.str.strip()
df.rename(columns={
    'Correo Cliente': 'correo_cliente',  # Solo se renombra 'Correo Cliente'
}, inplace=True)

# 2) Limpiar columna 'documento_cliente'
df['documento_cliente'] = (
    df['documento_cliente']
    .astype(str)
    .str.replace(r'[^\d]', '', regex=True)  # Quitar caracteres no numéricos
    .replace('', '0')                       # Convertir vacíos en '0'
)

# 3) Limpiar la columna 'numero_telefono' y reemplazar vacíos por NULL
df['numero_telefono'] = df['numero_telefono'].astype(str).str.strip().replace('', 'NULL')

# 4) Extraer solo filas únicas, tomando las columnas relevantes
df_clientes = df[['documento_cliente', 'nombre_cliente', 'numero_telefono', 'correo_cliente']].drop_duplicates()

# 5) Conexión a la base de datos y vaciar la tabla si es necesario
try:
    conn = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        'SERVER=localhost;'
        'DATABASE=quick;'
        'Trusted_Connection=yes;'
    )
    cursor = conn.cursor()
except Exception as e:
    print(f"Error al conectar con la base de datos: {e}")
    exit(1)

# 6) Inserción de los datos con verificación de existencia
clientes_insertados = 0
for i, row in df_clientes.iterrows():
    try:
        # Verificar si el cliente ya existe en la base de datos
        cursor.execute("""
            SELECT COUNT(*) FROM Clientes WHERE documento_cliente = ?
        """, row.documento_cliente)

        if cursor.fetchone()[0] == 0:  # Si no existe, insertamos el nuevo cliente
            correo = row.correo_cliente.strip() if isinstance(row.correo_cliente, str) else ''
            telefono = row.numero_telefono.strip() if isinstance(row.numero_telefono, str) else 'NULL'  # Asignar NULL si está vacío
            nombre_cliente = row.nombre_cliente.strip() if isinstance(row.nombre_cliente, str) else ''

            # Si 'telefono' es 'NULL', pasarlo como None para que SQL lo interprete como NULL
            cursor.execute("""
                INSERT INTO Clientes (documento_cliente, nombre_cliente, numero_telefono, correo_cliente)
                VALUES (?, ?, ?, ?)
            """, row.documento_cliente, nombre_cliente, telefono if telefono != 'NULL' else None, correo)

            clientes_insertados += 1

            # Mostrar progreso cada 50 registros
            if (i + 1) % 50 == 0:
                clear_output(wait=True)
                print(f"Ingresados: {i + 1} clientes...")

    except Exception as e:
        print(f"Error al procesar el cliente {row.documento_cliente}: {e}")

# Confirmar y hacer commit
conn.commit()

# 7) Verificación del número de clientes cargados
print(f"Clientes cargados: {clientes_insertados}")

# Cerrar la conexión
conn.close()






Ingresados: 50 clientes...
Clientes cargados: 272
